### Hugging Face AI Project


In [2]:
# Install Shap-e from github
!pip install transformers accelerate -q
!pip install git+https://github.com/openai/shap-e

  Cloning https://github.com/openai/shap-e to /private/var/folders/2j/5_hr8sgx70ndstdm_g1l4qx00000gn/T/pip-req-build-na0vkq01
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e /private/var/folders/2j/5_hr8sgx70ndstdm_g1l4qx00000gn/T/pip-req-build-na0vkq01
  Resolved https://github.com/openai/shap-e to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/2j/5_hr8sgx70ndstdm_g1l4qx00000gn/T/pip-install-3mf744_w/clip_13c1cefc7fc84164b041d5029e4bc068
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/2j/5_hr8sgx70ndstdm_g1l4qx00000gn/T/pip-install-3mf744_w/clip_13c1cefc7fc84164b041d5029e4bc068
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)


In [4]:
pip install transformers accelerate diffusers -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import diffuser pipelien

from diffusers import ShapEPipeline
from diffusers.utils import export_to_gif

In [6]:
# import pytorch for ML tasks
import torch
# import neccessary packages from shap-e
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [7]:
# selecting pre trained model and giving it object name
ckpt_id = "openai/shap-e"
# defining hardware for image generation. wants to use GPU but if non available uses CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# 3D renderer
xm = load_model('transmitter', device=device)

# text to 3D AI component
model = load_model('text300M', device=device)

# Diffusion process
diffusion = diffusion_from_config(load_config('diffusion'))

/Applications/anaconda3/lib/python3.12/site-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/Applications/anaconda3/lib/python3.12/site-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/Applications/anaconda3/lib/python3.12/site-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/Applications/anaconda3/lib/python3.12/site-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [01:02<00:00, 15.0MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

## Parameter and model control

In [ ]:


batch_size = 1 # how many versions of the 3-D object to create
guidance_scale = 16.0 # higher number -> less creativity, lower number -> gives AI more freedom
prompt = "Brown Wooden Club" # what 3-D object you would like to be rendered

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True, # change to false for higher quality, slower generation
    use_karras=True, # True is higher quality noise creation
    karras_steps=64, # higher -> more diffusion steps and creates higher quality but slower. bit increments (2, 4, 8, 16, 32, 64)
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf' stf is faster but lower quality
size = 128 # this is the size of the renders; higher values take longer to render.  bit increments (2, 4, 8, 16, 32, 64)

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)